## 一、赛题数据

赛题以预测二手车的交易价格为任务，数据集报名后可见并可下载，该数据来自某交易平台的二手车交易记录，总数据量超过40w，包含31列变量信息，其中15列为匿名变量。为了保证比赛的公平性，将会从中抽取15万条作为训练集，5万条作为测试集A，5万条作为测试集B，同时会对name、model、brand和regionCode等信息进行脱敏。


train.csv
字段表
- SaleID - 销售样本ID
- name - 汽车编码
- regDate - 汽车注册时间
- model - 车型编码
- brand - 品牌
- bodyType - 车身类型
- fuelType - 燃油类型
- gearbox - 变速箱
- power - 汽车功率
- kilometer - 汽车行驶公里
- notRepairedDamage - 汽车有尚未修复的损坏
- regionCode - 看车地区编码
- seller - 销售方
- offerType - 报价类型
- creatDate - 广告发布时间
- price - 汽车价格
- v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' 【匿名特征，包含v0-14在内15个匿名特征】 　
数字全都脱敏处理，都为label encoding形式，即数字形式


Field	Description
- SaleID	交易ID，唯一编码
- name	汽车交易名称，已脱敏
- regDate	汽车注册日期，例如20160101，2016年01月01日
- model	车型编码，已脱敏
- brand	汽车品牌，已脱敏
- bodyType	车身类型：豪华轿车：0，微型车：1，厢型车：2，大巴车：3，敞篷车：4，双门汽车：5，商务车：6，搅拌车：7
- fuelType	燃油类型：汽油：0，柴油：1，液化石油气：2，天然气：3，混合动力：4，其他：5，电动：6
- gearbox	变速箱：手动：0，自动：1
- power	发动机功率：范围 [ 0, 600 ]
- kilometer	汽车已行驶公里，单位万km
- notRepairedDamage	汽车有尚未修复的损坏：是：0，否：1
- regionCode	地区编码，已脱敏
- seller	销售方：个体：0，非个体：1
- offerType	报价类型：提供：0，请求：1
- creatDate	汽车上线时间，即开始售卖时间
- price	二手车交易价格（预测目标）
- v系列特征	匿名特征，包含v0-14在内15个匿名特征

## 二、评测标准

MAE(Mean Absolute Error)

什么是评估指标：

>评估指标即是我们对于一个模型效果的数值型量化。（有点类似与对于一个商品评价打分，而这是针对于模型效果和理想效果之间的一个打分）

#### 分类算法常见的评估指标如下：
* 对于二类分类器/分类算法，评价指标主要有accuracy， [Precision，Recall，F-score，Pr曲线]，ROC-AUC曲线。
* 对于多类分类器/分类算法，评价指标主要有accuracy， [宏平均和微平均，F-score]。

#### 对于回归预测类常见的评估指标如下:
* 平均绝对误差（Mean Absolute Error，MAE），均方误差（Mean Squared Error，MSE），平均绝对百分误差（Mean Absolute Percentage Error，MAPE），均方根误差（Root Mean Squared Error）， R2（R-Square）

平均绝对误差（Mean Absolute Error，MAE）:平均绝对误差，其能更好地反映预测值与真实值误差的实际情况，其计算公式如下：

$$
MAE=\frac{1}{N} \sum_{i=1}^{N}\left|y_{i}-\hat{y}_{i}\right|
$$

## 三丶结果提交

提交前请确保预测结果的格式与sample_submit.csv中的格式一致，以及提交文件后缀名为csv。

形式如下：

```
SaleID,price
150000,687
150001,1250
150002,2580
150003,1178

```

## 赛题初步分析

通过数据科学以及机器学习深度学习的办法来进行建模得到结果。

一个回归问题

主要应用xgb、lgb、catboost，以及pandas、numpy、matplotlib、seabon、sklearn、keras等等数据挖掘常用库或者框架来进行数据挖掘任务。

通过EDA来挖掘数据的联系和自我熟悉数据。


In [1]:
import pandas as pd
import numpy as np

path = '/Users/ousei/Documents/数据分析/study/data/'
Train_data = pd.read_csv(path+'car_train_0110.csv',sep=' ')
Test_data = pd.read_csv(path+'car_testA_0110.csv',sep=' ')

print('Train data shape:',Train_data.shape)
print('TestA data shape:',Test_data.shape)

Train data shape: (250000, 40)
TestA data shape: (50000, 39)


In [2]:
Train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_14,v_15,v_16,v_17,v_18,v_19,v_20,v_21,v_22,v_23
0,134890,734,20160002,13.0,9,NaN,0.0,1.0,0,15.0,...,0.092139,0.000000,18.763832,-1.512063,-1.008718,-12.100623,-0.947052,9.077297,0.581214,3.945923
1,306648,196973,20080307,72.0,9,7.0,5.0,1.0,173,15.0,...,0.001070,0.122335,-5.685612,-0.489963,-2.223693,-0.226865,-0.658246,-3.949621,4.593618,-1.145653
2,340675,25347,20020312,18.0,12,3.0,0.0,1.0,50,12.5,...,0.064410,0.003345,-3.295700,1.816499,3.554439,-0.683675,0.971495,2.625318,-0.851922,-1.246135
3,57332,5382,20000611,38.0,8,7.0,0.0,1.0,54,15.0,...,0.069231,0.000000,-3.405521,1.497826,4.782636,0.039101,1.227646,3.040629,-0.801854,-1.251894
4,265235,173174,20030109,87.0,0,5.0,5.0,1.0,131,3.0,...,0.000099,0.001655,-4.475429,0.124138,1.364567,-0.319848,-1.131568,-3.303424,-1.998466,-1.279368


## 分类指标评价与回归指标评价计算

## 前辈的经验

对于赛题的理解甚至会影响后续的特征工程构建以及模型的选择，最主要是会影响后续发展工作的方向，比如挖掘特征的方向或者存在问题解决问题的方向，对了赛题背后的思想以及赛题业务逻辑的清晰，也很有利于花费更少时间构建更为有效的特征模型，赛题理解要达到的地步是什么呢，把一道赛题转化为一种宏观理解的解决思路。

理解赛题其实也是从直观上梳理问题，分析问题是否可行的方法，有多少可行度，赛题做的价值大不大，理清一道赛题要从背后的赛题背景引发的赛题任务理解其中的任务逻辑，可能对于赛题有意义的外在数据有哪些，并对于赛题数据有一个初步了解，知道现在和任务的相关数据有哪些，其中数据之间的关联逻辑是什么样的。

大概要去用哪些指标，哪些指标是否会做到线上线下的一致性，是否有效的利于我们进一步的探索更高线上分数的线下验证方法，在业务上，你是否对很多原始特征有很深刻的了解，并且可以通过EDA来寻求他们直接的关系，最后构造出满意的特征。

这题的难点可能在哪里，关键点可能在哪里，哪些地方可以挖掘更好的特征，用什么样得线下验证方式更为稳定，出现了过拟合或者其他问题，估摸可以用什么方法去解决这些问题，哪些数据是可靠的，哪些数据是需要精密的处理的，哪部分数据应该是关键数据（背景的业务逻辑下，比如CTR的题，一个寻常顾客大体会有怎么样的购买行为逻辑规律，或者风电那种题，如果机组比较邻近，相关一些风速，转速特征是否会很近似）。

1.在比赛中构建一个合理的本地的验证集和验证的评价指标是很关键的步骤，能有效的节省很多时间。

2.不同的指标对于同样的预测结果是具有误差敏感的差异性的，比如AUC，logloss, MAE，RSME，或者一些特定的评价函数。是会有很大可能会影响后续一些预测的侧重点。

其实赛题中有些说明是很有利益-都可以在后续答辩中以及问题思考中所体现出来的，比如高效性要求，比如对于数据异常的识别处理，比如工序流程的差异性，比如模型运行的时间，比模型的鲁棒性，有些的意识是可以贯穿问题思考，特征，模型以及后续处理的，也有些会对于特征构建或者选择模型上有很大益处，反过来如果在模型预测效果不好，其实有时也要反过来思考，是不是赛题背景有没有哪方面理解不清晰或者什么其中的问题没考虑到。